In [1]:
import cv2 
import numpy as np
import csv
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
import keras
print(keras.__version__)


lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
images = []
measurements = []
for line in lines:
    if line[0] == 'center':
        continue
    
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
X_train = np.array(images)
y_train = np.array(measurements)


model = Sequential()
model.add(Lambda(lambda x: x / 255.0 -0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss = 'mse', optimizer='adam')
print('Printing...')
model.fit(X_train, y_train, validation_split = 0.2, shuffle=True, nb_epoch=2)
model.save('model.h5')
print('DOne')



Using TensorFlow backend.


2.2.2
Printing...
Train on 6428 samples, validate on 1608 samples
Epoch 1/2


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:38: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


6428/6428 [==============================] - 11s 2ms/step - loss: 1.6655 - val_loss: 0.7797
Epoch 2/2
6428/6428 [==============================] - 8s 1ms/step - loss: 3.2393 - val_loss: 2.5561
DOne
